# Descrição do problema

Usuários da eNB1 se afastam da mesma em linha reta e com um ângulo aleatório, enquanto realizam o download de um vídeo de 15 MB. Eventualmente, devido ao enfraquecimento do sinal da sua eNB, eles fazem handover para a eNB2 ou eNB3 através do algoritmo A3RSRP. Com acesso aos **sinais de referência** das 3 eNBs no momento do handover, use redes neurais para prever em qual **região** (region) se encontra cada usuário e qual **vazão** (throughput) possuem os mesmos nos primeiros 100 segundos de simulação.
<img src="ang.png" style="width: 400px">

# Parte 1 - Pré-processamento de dados

<font color=blue>Nesta etapa do HandsOn, você irá pré-processar os dados coletados de forma que os mesmos estejam adequados à utilização por redes neurais.</font>

## Importando o dataset e as bibliotecas essenciais

In [7]:
# Biblioteca usada para criar o frame de dados
import pandas as pd
# Importando a biblioteca base do python
import numpy as np

# Ignorando warnings (opcional)
import warnings
warnings.filterwarnings("ignore")

# Importando os dados
data=pd.read_csv('data.txt', delimiter='\t')

# Visualizando as primeiras linhas da variável data
data.head()

,nRun,rsrp1,rsrq1,rsrp2,rsrq2,rsrp3,rsrq3,region,throughput
0,0,-97.9726,-9.52930,-97.1633,-8.72002,-94.3961,-5.95282,superior,3.956754
1,1,-96.3151,-8.53851,-92.5126,-4.73598,-104.0420,-16.26500,inferior,3.950170
2,2,-96.7161,-8.88537,-92.9135,-5.08275,-100.0370,-12.20660,central,3.049065
3,3,-96.9771,-9.01505,-93.1471,-5.18508,-99.4380,-11.47600,central,2.912435
4,4,-96.2499,-8.68470,-92.2736,-4.70835,-103.3940,-15.82840,inferior,3.803656


Obs.: O **throughput** se encontra em Mbps.

## Dividindo os dados em variáveis independentes e dependente
* A primeira tarefa a ser desempenhada pela rede neural será tentar prever em qual região o usuário que gerou esses dados está. Desta forma, *region* será a variável dependente.
* Deve-se tentar identificar quais variáveis independentes influenciam no valor da região, pois apenas estas variáveis devem ser passadas para a rede.
   * Resposta: os valores dos sinais de referência que ajudam a localizar a posiçao do usuário em relacao às 3 eNBs .

In [2]:
# Variáveis independentes
x=data.iloc[:,1:7]
print('Variáveis independentes\n'+str(x.head())+'\n')
# Variável dependente
y=data.iloc[:,7]
print('Variável dependente\n'+str(y.head()))

Variáveis independentes
     rsrp1    rsrq1    rsrp2    rsrq2     rsrp3     rsrq3
0 -97.9726 -9.52930 -97.1633 -8.72002  -94.3961  -5.95282
1 -96.3151 -8.53851 -92.5126 -4.73598 -104.0420 -16.26500
2 -96.7161 -8.88537 -92.9135 -5.08275 -100.0370 -12.20660
3 -96.9771 -9.01505 -93.1471 -5.18508  -99.4380 -11.47600
4 -96.2499 -8.68470 -92.2736 -4.70835 -103.3940 -15.82840

Variável dependente
0    superior
1    inferior
2     central
3     central
4    inferior
Name: region, dtype: object


## Lidando com variáveis categóricas
* No quadro acima, a variável *region* possui 3 categorias: superior, inferior e central. 
* O algoritmo não saberá lidar com essa variável em formato textual, portanto, é necessário expressá-la em números.

In [35]:
# Importando a biblioteca usada para codificar variáveis categóricas
from sklearn.preprocessing import LabelEncoder

# Criando um objeto Label Encoder
label_encoder=LabelEncoder()

# Transformando os valores da varável region
y=label_encoder.fit_transform(y)

# Resultado
print(y)

[2 1 0 ... 1 1 1]


### Variáveis categóricas com mais de 2 categorias e sem distição de maior-menor 
* A variável *region* tem agora suas categorias indicadas pelos números 0, 1 e 2. 
* Isto pode levar errôneamente a maquina a inferir que a região superior tem um valor maior que a central, que por sua vez, tem um valor maior do que a inferior, atrapalhado o processamento dos dados.
* Para evitar este erro, devemos usar variáveis "burras" (**dummy variables**).
* Isto não acontece quando há apenas 2 categorias.

In [36]:
# Importando a biblioteca usada para codificar as variáveis (já numericamente categorizadas) em dummy variables 
from sklearn.preprocessing import OneHotEncoder

# Alterando a forma de y para várias linhas e 1 coluna
y=y.reshape(-1, 1)

# Criando um objeto One Hot Encoder
onehotencoder = OneHotEncoder()
y = onehotencoder.fit_transform(y).toarray()

# Resultado
print(y)

[[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


## Dividindo os dados em conjunto de treino e conjunto de testes
* Antes de estar pronta para fazer suas previsões, a rede precisa passar por um **processo de aprendizagem** para se ajustar ao problema em questão. Neste caso, nossa rede aprenderá de maneira **supervisionada**, o que significa que forneceremos vários exemplos (**conjunto de treino**) com as respostas esperadas para que a rede **ajuste seus pesos**.
* Após o processo de aprendizagem, podemos usar a rede para prever a informação desejada. Usualmente, testamos a eficácia da rede em um **conjunto de testes**, que é composto por exemplos que não estão presentes no conjunto de treino. Neste estágio, a rede apenas usa o que foi aprendido no anterior, sem reajustar seus pesos. As respostas esperadas não são, portanto, passadas à rede, sendo apenas usadas para medir a performance da mesma.
* A biblioteca sklearn possui uma função que divide os dados da maneira mais eficaz possível, garantindo que os conjuntos de treino e testes possuam, ambos, amostras que representem de forma satisfatória todo o dataset. Já imaginou o que aconteceria se todos os exemplos passados à rede na etapa de treino fossem da mesma região? A rede jamais aprenderia a identificar as demais e sua performance seria bem limitada. Da mesma forma, não teríamos uma dimensao exata da precisão da rede se a testássemos apenas para uma região. Por isso, é importante usar a função ***train_test_split***.

In [37]:
# Importando a funcao train_test_split
from sklearn.model_selection import train_test_split

# Obtendo os conjuntos de treino (80%) e testes (20%)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=0)

In [38]:
# Verificando a quantidade de dados de cada regiao

# Conjunto de treino
inf=y_train[:,0].sum()
cent=y_train[:,1].sum()
sup=y_train[:,2].sum()
print('Composicao do conjunto de treino:')
print('Regiao inferior = '+str(inf))
print('Regiao central = '+str(cent))
print('Regiao superior = '+str(sup) + '\n')

# Conjunto de testes
inf=y_test[:,0].sum()
cent=y_test[:,1].sum()
sup=y_test[:,2].sum()
print('Composicao do conjunto de testes:')
print('Regiao inferior = '+str(inf))
print('Regiao central = '+str(cent))
print('Regiao superior = '+str(sup))

Composicao do conjunto de treino:
Regiao inferior = 2598.0
Regiao central = 2706.0
Regiao superior = 2696.0

Composicao do conjunto de testes:
Regiao inferior = 675.0
Regiao central = 645.0
Regiao superior = 680.0


## Feature scaling
* É importante deixar os **features** (variáveis independentes) na **mesma escala** para acelerar o processamento da rede neural e melhorar o seu desempenho.
* Existem vários tipos de padronizadores de escala para python, usaremos aqui o StandardScaler e o MinMaxScaler.

### *StandardScaler*
* Deixa cada feature de x com uma distribuição gaussiana centrada em 0 e com desvio padrão igual a 1.
* Fórmula:   
   #### <center> $ x_{stand} = \frac{x - media(x)}{std(x)}$ </center>

In [66]:
# Importando a biblioteca StandardScaler
from sklearn.preprocessing import StandardScaler

# Criando um objeto standard scaler
fs = StandardScaler()

# Ajustando fs ao conjunto de treino
# transformando os features do conjunto de treino
x_train = fs.fit_transform(x_train)

# Transformando os features do conjunto de testes
# Só é necessário usar "fit" no conjunto de treino
x_test = fs.transform(x_test)

# Resultado
print(x_test[0:4,:])

[[ 1.16792464  1.03977203  0.8876307   0.7611939  -0.98160775 -1.03671221]
 [ 0.63633682  0.40519077  0.81193876  0.72903243 -0.60431154 -0.64843178]
 [ 0.90296154  1.38392602  0.78732034  0.79062354 -1.51098648 -1.45141723]
 [-0.41584662 -0.22320138 -1.29457875 -1.26367518  1.19815723  1.19007538]]


### *MinMaxScaler*
* Também é conhecido como min-max normalizer.
* Deixa cada feature de x com valores percentuais na forma decimal, sendo min(x) = 0 e max(x) = 1.
* Fórmula:  
    ####  <center>  $ x_{norm} = \frac{x - min(x)}{max(x) - min(x)}$  </center> 


## Exercício 1
Realize o pré-processamento dos dados para encontrar a vazão (**throughput**) da rede com o padronizador *StandardScaler*.

## Exercício 2
No mesmo notebook do exercício 1, desfaça a padronização feita com o *StandardScaler* e repadronize os dados usando o *MinMaxScaler*.

Obs.: Para fazer esse exercício, você deverá consultar a documentação das classes <a href="http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html">StandardScaler</a> e <a href="http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html">MinMaxScaler</a>.